In [1]:
import pandas as pd
import polars as pl
from data import *
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import pyarrow 
import sklearn
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import os
import numpy as np
from scipy import stats
from typing import cast
from utils import *
import wrds

In [2]:
db = wrds.Connection(
    wrds_username='jayati'
)


Loading library list...
Done


In [3]:
full_data_imputed = pl.read_parquet("data/3_2_imputed_dataset.parquet")

In [4]:
full_data_imputed.height

5001719

In [7]:
crsp_monthly_query = """
SELECT msf.permno, msf.date, ssih.siccd
FROM crsp.msf AS msf
LEFT JOIN crsp.stksecurityinfohist AS ssih
ON msf.permno = ssih.permno
AND ssih.secinfostartdt <= msf.date AND msf.date <= ssih.secinfoenddt
WHERE msf.date BETWEEN '1925-12-01' AND '2023-12-01'
"""
crsp_monthly_data = db.raw_sql(crsp_monthly_query)

crsp_monthly_df = pl.from_pandas(crsp_monthly_data)

crsp_monthly_df.select(['permno', 'date', 'siccd']).write_csv('data/crsp_monthly_with_industries.csv')


In [8]:
full_data_imputed.head()

permno,yyyymm,date,date_right,AM,Accruals,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BidAskSpread,BookLeverage,CF,CashProd,ChEQ,ChInv,ChInvIA,ChNNCOA,ChNWC,ChTax,ConvDebt,CoskewACX,Coskewness,DebtIssuance,DelCOA,DelCOL,DelEqu,DelFINL,DelLTI,DelNetFin,DivInit,DivOmit,DolVol,…,NOA,NetDebtFinance,NetEquityFinance,NumEarnIncrease,OPLeverage,PctAcc,PriceDelayRsq,PriceDelaySlope,PriceDelayTstat,RDIPO,RealizedVol,ResidualMomentum,ReturnSkew,ReturnSkew3F,RoE,SP,ShareIss1Y,ShareIss5Y,ShareRepurchase,Spinoff,Tax,TotalAccruals,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dNoa,hire,zerotrade,zerotradeAlt1,zerotradeAlt12,STreversal,Price,Size,ret
i64,i64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
10000,198601,"""1986-01-01""","""1986-01-31""",16.202734,0.001839,-0.035723,-0.083313,0.21025,1.404658,0.225389,1.025208,0.680467,0.121024,0.019758,-2.326463,0.155227,26.920122,-1.14218,-0.009633,0.135743,-0.005331,-0.00193,-0.00254,-0.1,0.031704,0.208924,-0.7,-0.025725,-0.004444,-0.069119,-0.026367,-0.0165,-0.021856,0.0,0.0,2.203245,…,-0.618638,-0.010772,-0.007077,0.0,0.404598,0.178823,0.843958,-2.272196,-0.249299,0.0,-0.065278,-0.026406,-2.23832,-2.103353,0.059981,2.454645,-0.040535,-0.148483,0.3,0.0,0.760982,-0.017584,-0.027994,-1.680641,0.023985,-0.03372,-0.005234,0.069862,-0.047715,0.0,6.591505,6.951966,5.942311,0.0,-1.475907,-9.686575,0.707317
10000,198602,"""1986-02-01""","""1986-02-28""",6.419575,0.023401,-0.024668,-0.053147,-0.578258,1.615003,-0.074305,1.575849,0.897772,0.237806,0.026047,-5.2407,-0.029837,59.399372,-1.196283,0.027234,1.103777,-0.010218,0.003149,-0.019136,-0.1,0.138626,0.213954,-0.7,-0.024757,-0.003946,-0.047217,-0.007605,-0.019091,-0.000286,0.0,0.0,0.619522,…,-0.551703,-0.010474,-0.02755,0.0,0.408773,1.44153,0.874593,-0.377262,0.373869,0.0,-0.031004,0.240515,0.729555,0.810187,-0.269592,2.172496,-0.35988,-1.540913,0.2,0.0,0.9,-0.000392,-0.080506,-0.652343,-0.016094,-0.046509,-0.003488,0.16898,-0.063516,0.0,0.69714,4.7852e-8,0.967149,25.7143,-1.178655,-9.389323,-0.257143
10000,198603,"""1986-03-01""","""1986-03-31""",9.672034,0.01986,0.028711,-0.298639,-0.365597,0.83168,-0.471459,1.121976,0.7307,0.379325,0.017515,-3.163787,0.103764,40.948347,-1.24629,0.00244,1.388678,0.011832,-0.005521,-0.002551,0.0,0.065951,0.13381,-0.5,-0.025243,-0.018863,-0.069383,-0.035775,-0.016708,-0.04417,0.0,0.0,0.255134,…,-0.504815,-0.021901,-0.086281,0.0,0.166429,3.065052,0.832632,5.132098,3.2198343,0.0,-0.044548,0.30884,-1.6879,-1.038325,-0.012496,1.240049,-0.142092,-1.21123,0.2,0.0,0.82922,-0.004646,-0.048295,-0.203063,-0.010171,-0.168674,-0.002715,0.256035,-0.089197,0.0,0.522273,1.0234e-7,1.036358,-36.5385,-1.490091,-9.700759,0.365385
10000,198604,"""1986-04-01""","""1986-04-30""",34.015608,0.005994,-0.02554,-0.086369,0.401779,3.266376,-2.110798,1.148184,0.708417,-0.03177,0.01237,-2.37108,0.264075,17.657581,-1.178537,-0.005092,0.852608,-0.004871,-0.00011,-0.006269,0.0,0.149604,0.326355,-0.5,-0.029283,-0.006735,-0.070258,-0.030067,-0.017173,-0.035524,0.0,0.0,1.3126723,…,-0.6425,-0.023357,0.000215,0.0,0.413942,0.188181,0.811953,-2.28961,-0.412962,0.0,-0.011246,0.097471,0.276167,0.068097,0.144804,6.422166,0.088618,-0.37541,0.2,0.0,0.853708,-0.015898,-0.039136,-2.814366,0.00728,-0.011929,0.000877,0.084696,-0.051597,0.0,2.197636,7.4675e-8,2.679387,9.8592,-1.386294,-9.627207,-0.098592
10000,198605,"""1986-05-01""","""1986-05-30""",8.585498,0.012495,-0.045491,-0.016131,-0.351601,2.188701,-0.689932,1.625661,1.082514,0.110916,0.023352,-4.313698,0.00277,31.042189,-1.03832,0.030501,-0.817816,-0.013207,-0.006578,-0.004206,-0.1,0.201706,0.488893,-0.7,-0.017447,0.000681,-0.038604,-0.020003,-0.003672,-0.01622,0.0,0.0,0.737386,…,-0.457639,-0.020878,-0.007652,0.0,0.269973,4.040911,0.829054,1

In [9]:
crsp_monthly_df.head()

permno,date,siccd
i64,date,f64
10000,1985-12-31,null
10000,1986-01-31,3990.0
10000,1986-02-28,3990.0
10000,1986-03-31,3990.0
10000,1986-04-30,3990.0


In [10]:
crsp_monthly_df.height

5027863

In [11]:
full_data_imputed = full_data_imputed.with_columns([
    pl.col('permno').cast(pl.Int64),
    pl.col('date').cast(pl.Date)
])

crsp_monthly_df = crsp_monthly_df.with_columns([
    pl.col('permno').cast(pl.Int64),
    pl.col('date').cast(pl.Date)
])

# Extract year and month from the date columns
full_data_imputed = full_data_imputed.with_columns([
    pl.col('date').dt.year().alias('year'),
    pl.col('date').dt.month().alias('month')
])

crsp_monthly_df = crsp_monthly_df.with_columns([
    pl.col('date').dt.year().alias('year'),
    pl.col('date').dt.month().alias('month')
])

# Perform the join operation on permno, year, and month
full_data_imputed = full_data_imputed.join(
    crsp_monthly_df.select(['permno', 'year', 'month', 'siccd']),
    on=['permno', 'year', 'month'],
    how='left'
)

print(full_data_imputed.head())

shape: (5, 103)
┌────────┬────────┬────────────┬────────────┬───┬───────────┬──────┬───────┬────────┐
│ permno ┆ yyyymm ┆ date       ┆ date_right ┆ … ┆ ret       ┆ year ┆ month ┆ siccd  │
│ ---    ┆ ---    ┆ ---        ┆ ---        ┆   ┆ ---       ┆ ---  ┆ ---   ┆ ---    │
│ i64    ┆ i64    ┆ date       ┆ str        ┆   ┆ f64       ┆ i32  ┆ i8    ┆ f64    │
╞════════╪════════╪════════════╪════════════╪═══╪═══════════╪══════╪═══════╪════════╡
│ 10000  ┆ 198601 ┆ 1986-01-01 ┆ 1986-01-31 ┆ … ┆ 0.707317  ┆ 1986 ┆ 1     ┆ 3990.0 │
│ 10000  ┆ 198602 ┆ 1986-02-01 ┆ 1986-02-28 ┆ … ┆ -0.257143 ┆ 1986 ┆ 2     ┆ 3990.0 │
│ 10000  ┆ 198603 ┆ 1986-03-01 ┆ 1986-03-31 ┆ … ┆ 0.365385  ┆ 1986 ┆ 3     ┆ 3990.0 │
│ 10000  ┆ 198604 ┆ 1986-04-01 ┆ 1986-04-30 ┆ … ┆ -0.098592 ┆ 1986 ┆ 4     ┆ 3990.0 │
│ 10000  ┆ 198605 ┆ 1986-05-01 ┆ 1986-05-30 ┆ … ┆ -0.222656 ┆ 1986 ┆ 5     ┆ 3990.0 │
└────────┴────────┴────────────┴────────────┴───┴───────────┴──────┴───────┴────────┘


In [12]:
sic_groups = [
    ('Agriculture, Forestry, and Fishing', range(100, 1000)),
    ('Mining', range(1000, 1500)),
    ('Construction', range(1500, 1800)),
    ('Manufacturing', range(2000, 4000)),
    ('Transportation, Communications, Electric, Gas, and Sanitary Services', range(4000, 5000)),
    ('Wholesale Trade', range(5000, 5200)),
    ('Retail Trade', range(5200, 6000)),
    ('Finance, Insurance, and Real Estate', range(6000, 6800)),
    ('Services', range(7000, 9000)),
    ('Public Administration', range(9000, 10000))
]

# Flatten the SIC groups into a dictionary for fast lookup
sic_dict = {}
for industry, sic_range in sic_groups:
    for sic in sic_range:
        sic_dict[sic] = industry

# Create a Polars expression to map the SIC codes to industries
def map_sic_to_industry(sic):
    return sic_dict.get(sic, 'Unknown')

# Apply the mapping and count the number of stocks in each industry group
industry_counts = (full_data_imputed
                   .with_columns(pl.col('siccd').map_elements(map_sic_to_industry, return_dtype=pl.Utf8).alias('industry'))
                   .group_by('industry')
                   .agg(pl.count('permno').alias('count')))

print(industry_counts)


shape: (12, 2)
┌─────────────────────────────────┬─────────┐
│ industry                        ┆ count   │
│ ---                             ┆ ---     │
│ str                             ┆ u32     │
╞═════════════════════════════════╪═════════╡
│ Services                        ┆ 512483  │
│ Retail Trade                    ┆ 250349  │
│ Mining                          ┆ 261277  │
│ Agriculture, Forestry, and Fis… ┆ 13619   │
│ Finance, Insurance, and Real E… ┆ 1348191 │
│ …                               ┆ …       │
│ Transportation, Communications… ┆ 386362  │
│ Construction                    ┆ 43816   │
│ Public Administration           ┆ 165450  │
│ Wholesale Trade                 ┆ 136772  │
│ Manufacturing                   ┆ 1745782 │
└─────────────────────────────────┴─────────┘


In [18]:
full_data_industries = full_data_imputed.with_columns(
    pl.col('siccd').map_elements(map_sic_to_industry).alias('industry')
)

/var/folders/b0/4ztxyrhx5zb29cnt86sf6m580000gn/T/ipykernel_36610/1437896745.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  full_data_industries = full_data_imputed.with_columns(


In [19]:
full_data_imputed.height

5001719

In [24]:
# Ensure the date column is of the correct type
full_data_reduced = full_data_industries.with_columns([
    pl.col('date').cast(pl.Date)
])

# Filter out rows with dates before 1990
full_data_reduced = full_data_reduced.filter(pl.col('date').dt.year() >= 2000)


print(full_data_reduced.head())

shape: (5, 104)
┌────────┬────────┬────────────┬────────────┬───┬──────┬───────┬────────┬─────────────────┐
│ permno ┆ yyyymm ┆ date       ┆ date_right ┆ … ┆ year ┆ month ┆ siccd  ┆ industry        │
│ ---    ┆ ---    ┆ ---        ┆ ---        ┆   ┆ ---  ┆ ---   ┆ ---    ┆ ---             │
│ i64    ┆ i64    ┆ date       ┆ str        ┆   ┆ i32  ┆ i8    ┆ f64    ┆ str             │
╞════════╪════════╪════════════╪════════════╪═══╪══════╪═══════╪════════╪═════════════════╡
│ 10001  ┆ 200001 ┆ 2000-01-01 ┆ 2000-01-31 ┆ … ┆ 2000 ┆ 1     ┆ 4920.0 ┆ Transportation, │
│        ┆        ┆            ┆            ┆   ┆      ┆       ┆        ┆ Communications… │
│ 10001  ┆ 200002 ┆ 2000-02-01 ┆ 2000-02-29 ┆ … ┆ 2000 ┆ 2     ┆ 4920.0 ┆ Transportation, │
│        ┆        ┆            ┆            ┆   ┆      ┆       ┆        ┆ Communications… │
│ 10001  ┆ 200003 ┆ 2000-03-01 ┆ 2000-03-31 ┆ … ┆ 2000 ┆ 3     ┆ 4920.0 ┆ Transportation, │
│        ┆        ┆            ┆            ┆   ┆      ┆       ┆

In [50]:
full_data_reduced=full_data_reduced.drop("year", "month")

In [51]:
full_data_reduced.head()

permno,yyyymm,date,date_right,AM,Accruals,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BidAskSpread,BookLeverage,CF,CashProd,ChEQ,ChInv,ChInvIA,ChNNCOA,ChNWC,ChTax,ConvDebt,CoskewACX,Coskewness,DebtIssuance,DelCOA,DelCOL,DelEqu,DelFINL,DelLTI,DelNetFin,DivInit,DivOmit,DolVol,…,NetEquityFinance,NumEarnIncrease,OPLeverage,PctAcc,PriceDelayRsq,PriceDelaySlope,PriceDelayTstat,RDIPO,RealizedVol,ResidualMomentum,ReturnSkew,ReturnSkew3F,RoE,SP,ShareIss1Y,ShareIss5Y,ShareRepurchase,Spinoff,Tax,TotalAccruals,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dNoa,hire,zerotrade,zerotradeAlt1,zerotradeAlt12,STreversal,Price,Size,ret,siccd,industry
i64,i64,date,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str
10001,200001,2000-01-01,"""2000-01-31""",2.1733375,0.108981,-0.033873,-0.019392,-0.445696,0.752592,-1.907572,0.103002,0.091242,0.016507,0.031007,-2.466956,0.164873,103.34845,-1.05628,0.074093,0.6538,0.014184,0.072073,0.000448,0.0,0.123057,0.073287,0.0,0.024853,-0.044573,-0.016826,0.043476,0.000187,0.043289,0.0,0.0,0.938377,…,0.02357,2.0,1.1216513,2.953371,0.772187,-4.625865,3.7699442,0.0,-0.025284,-0.293568,-0.049701,-0.007985,0.117278,2.6856389,-0.019559,-0.101619,0.0,0.0,0.478891,-0.009637,-0.023017,-0.025291,-0.013213,0.066246,-0.000971,0.315177,0.030867,-0.048,2.7886364,3.818182,3.1109962,4.4118,-2.094946,-9.898789,-0.044118,4920.0,"""Transportation, Communications…"
10001,200002,2000-02-01,"""2000-02-29""",2.1404083,0.108981,0.013153,-0.019392,-0.445696,0.752592,-1.778563,0.104569,0.086462,0.017932,0.017571,-2.466956,0.162375,101.99336,-1.05628,0.074093,0.648363,0.014184,0.072073,0.000448,0.0,0.07828,0.085667,0.0,0.024853,-0.044573,-0.016826,0.043476,0.000187,0.043289,0.0,0.0,1.2864491,…,0.02357,2.0,1.1216513,2.953371,0.772187,-4.625865,3.7699442,0.0,-0.012606,-0.18632,-0.645254,-0.593679,0.117278,2.6449475,-0.019559,-0.101619,0.0,0.0,0.478891,-0.009637,-0.022175,-0.024616,-0.011518,0.066246,-0.00774,0.310402,0.030867,-0.048,3.1863637,1.0500002,2.8300753,-1.5385,-2.110213,-9.914057,0.015385,4920.0,"""Transportation, Communications…"
10001,200003,2000-03-01,"""2000-03-31""",2.1947544,0.108981,0.013153,-0.019392,-0.445696,0.752592,-2.003696,0.107111,0.087634,0.009077,0.024509,-2.466956,0.166498,104.20796,-1.05628,0.074093,0.652208,0.014184,0.072073,-0.003487,0.0,0.089577,0.089791,0.0,0.024853,-0.044573,-0.016826,0.043476,0.000187,0.043289,0.0,0.0,1.1164581,…,0.02357,0.0,1.1216513,2.953371,0.772187,-4.625865,3.7699442,0.0,-0.026537,-0.120605,-0.232345,-0.22865,0.117278,2.7121043,-0.017865,-0.098039,0.0,0.0,0.478891,-0.009637,-0.02242,-0.024579,-0.010919,0.066246,-0.006041,0.318283,0.030867,-0.048,3.5994072,5.2500005,2.8070488,1.5758,-2.079442,-9.888983,-0.015288,4920.0,"""Transportation, Communications…"
10001,200004,2000-04-01,"""2000-04-28""",2.1693325,0.108981,0.013153,-0.019392,-0.445696,0.752592,-1.890749,0.096826,0.07906,-0.008956,0.024083,-2.466956,0.164569,103.18584,-1.05628,0.074093,0.665905,0.014184,0.072073,-0.003487,0.0,0.248881,0.060906,0.0,0.024853,-0.044573,-0.016826,0.043476,0.000187,0.043289,0.0,0.0,1.6974498,…,0.02357,0.0,1.1216513,2.953371,0.772187,-4.625865,3.7699442,0.0,-0.022609,-0.216381,-0.28195,-0.408123,0.117278,2.6806898,-0.017865,-0.098039,0.0,0.0,0.478891,-0.009637,-0.019742,-0.024804,-0.009217,0.066246,0.003357,0.314597,0.030867,-0.048,3.8362494,5.478261,3.0353098,-1.1719,-2.091092,-9.900633,0.011719,4920.0,"""Transportation, Communications…"
10001,200005,2000-05-01,"""2000-05-31""",2.2207792,0.108981,0.067251,-0.019392,-0.445696,0.752592,-2.136743,0.105629,0.068718,-0.011836,0.028469,-2.466956,0.168472,105.23009,-1.05628,0.074093,0.668498,0.014184,0.072073,-0.003487,0.0,0.272532,0.03478,0.0,0.024853,-0.044573,-0.016826,0.043476,0.000187,0.043289,0.

In [52]:
full_data_reduced.write_csv('data/Industries_all.csv')

In [53]:
industries = full_data_reduced.select(pl.col('industry')).unique().to_series().to_list()

os.makedirs('data/industry_datasets', exist_ok=True)

for industry in industries:
    if industry is not None:
        industry_data = full_data_reduced.filter(pl.col('industry') == industry)
        industry_filename = f"data/industry_datasets/{industry.replace(' ', '_')}_data.csv"
        industry_data.write_csv(industry_filename)
        print(f"Data for {industry} saved to {industry_filename}")


Data for Agriculture, Forestry, and Fishing saved to data/industry_datasets/Agriculture,_Forestry,_and_Fishing_data.csv
Data for Services saved to data/industry_datasets/Services_data.csv
Data for Finance, Insurance, and Real Estate saved to data/industry_datasets/Finance,_Insurance,_and_Real_Estate_data.csv
Data for Manufacturing saved to data/industry_datasets/Manufacturing_data.csv
Data for Transportation, Communications, Electric, Gas, and Sanitary Services saved to data/industry_datasets/Transportation,_Communications,_Electric,_Gas,_and_Sanitary_Services_data.csv
Data for Retail Trade saved to data/industry_datasets/Retail_Trade_data.csv
Data for Public Administration saved to data/industry_datasets/Public_Administration_data.csv
Data for Wholesale Trade saved to data/industry_datasets/Wholesale_Trade_data.csv
Data for Construction saved to data/industry_datasets/Construction_data.csv
Data for Mining saved to data/industry_datasets/Mining_data.csv
Data for Unknown saved to data/i

In [119]:
file = 'data/industry_datasets/Agriculture,_Forestry,_and_Fishing_data.csv'

In [120]:
data = pl.read_csv(file)
final = data.drop("industry", "siccd")
final.head()


permno,yyyymm,date,date_right,AM,Accruals,AnnouncementReturn,AssetGrowth,BM,BMdec,BPEBM,Beta,BetaFP,BetaLiquidityPS,BidAskSpread,BookLeverage,CF,CashProd,ChEQ,ChInv,ChInvIA,ChNNCOA,ChNWC,ChTax,ConvDebt,CoskewACX,Coskewness,DebtIssuance,DelCOA,DelCOL,DelEqu,DelFINL,DelLTI,DelNetFin,DivInit,DivOmit,DolVol,…,NOA,NetDebtFinance,NetEquityFinance,NumEarnIncrease,OPLeverage,PctAcc,PriceDelayRsq,PriceDelaySlope,PriceDelayTstat,RDIPO,RealizedVol,ResidualMomentum,ReturnSkew,ReturnSkew3F,RoE,SP,ShareIss1Y,ShareIss5Y,ShareRepurchase,Spinoff,Tax,TotalAccruals,VolMkt,VolSD,VolumeTrend,XFIN,betaVIX,cfp,dNoa,hire,zerotrade,zerotradeAlt1,zerotradeAlt12,STreversal,Price,Size,ret
i64,i64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
11614,200001,"""2000-01-01""","""2000-01-31""",1.9973766,-0.074983,0.004631,-0.386549,-0.370676,0.954144,0.261776,1.369328,0.755381,0.219421,0.025207,-1.522414,0.075995,197.90009,-2.003229,-0.080393,0.976063,0.083908,-0.042774,0.004482,0.0,0.215701,0.524991,-1.0,-0.150369,-0.046415,-0.356917,0.086197,0.0,0.080361,0.0,0.0,-2.646898,…,-1.202175,0.086197,-0.334449,2.0,0.376831,-4.889317,0.2705642,1.6056138,3.4281852,0.0,-0.044484,-0.087018,-2.492638,-2.999724,0.018555,0.8983103,-0.048813,-8.841158,0.0,0.0,0.452339,-0.412641,-0.095833,-1.373154,-0.020108,-0.213648,-0.018445,-0.088407,-0.327786,0.120577,2.2349e-8,1.1257e-8,2.7691e-8,-6.9167,-1.852384,-11.74086,0.070056
11614,200002,"""2000-02-01""","""2000-02-29""",2.7581959,-0.16542,0.004631,-0.414755,-0.102482,1.0202855,-1.78728,1.27067,0.78,0.213671,0.020525,-2.014216,0.114533,294.20328,-1.043433,-0.1758,0.776595,0.12408,-0.105743,0.000558,0.0,0.228408,0.516442,-1.0,-0.231268,-0.035137,-0.022064,-0.270667,0.0,-0.270667,0.0,0.0,-3.038993,…,-1.226052,-0.253642,0.008973,3.0,0.591458,-6.86595,0.2705642,1.6056138,3.4281852,0.0,-0.037149,-0.042604,0.807652,1.201192,0.034755,1.8405613,-0.030709,-8.841158,0.0,0.0,0.586278,-0.010866,-0.09052,-1.386931,-0.017669,-0.208804,0.007696,-0.25437,-0.352894,-0.668524,2.3455e-8,2.1190e-8,2.7819e-8,-2.451,-1.876598,-11.765074,0.02451
11614,200003,"""2000-03-01""","""2000-03-31""",3.3130052,-0.16542,0.004631,-0.414755,-0.102482,1.0202855,-1.763067,1.288584,0.744443,0.187754,0.022869,-2.014216,0.137572,322.22546,-1.043433,-0.1758,0.598506,0.12408,-0.105743,0.000558,0.0,0.245173,0.496407,-1.0,-0.231268,-0.035137,-0.022064,-0.270667,0.0,-0.270667,0.0,0.0,-2.562339,…,-1.226052,-0.253642,0.008973,3.0,0.591458,-6.86595,0.2705642,1.6056138,3.4281852,0.0,-0.053644,-0.24325,-0.017113,-0.617639,0.034755,2.210789,-0.030709,-8.826433,0.0,0.0,0.586278,-0.010866,-0.102077,-1.404163,-0.01529,-0.208804,0.008048,-0.305536,-0.352894,-0.668524,2.3826e-8,4.0763e-8,2.8828e-8,16.7464,-1.693319,-11.581795,-0.167464
11614,200004,"""2000-04-01""","""2000-04-28""",4.9695082,-0.16542,-0.109444,-0.414755,-0.102482,1.0202855,-2.391307,1.426398,0.723738,0.347908,0.0381,-2.014216,0.206357,368.66223,-1.043433,-0.1758,0.594765,0.12408,-0.105743,0.000558,0.0,0.25822,0.533109,-1.0,-0.231268,-0.035137,-0.022064,-0.270667,0.0,-0.270667,0.0,-1.0,-1.932317,…,-1.226052,-0.253642,0.008973,3.0,0.591458,-6.86595,0.2705642,1.6056138,3.4281852,0.0,-0.068779,-0.267446,-1.316595,-1.101947,0.034755,3.3161838,-0.030709,-8.826433,0.0,0.0,0.586278,-0.010866,-0.135278,-1.088585,-0.012911,-0.208804,0.002926,-0.458305,-0.352894,-0.668524,2.7163e-8,3.2372e-8,3.0045e-8,33.3333,-1.287854,-11.17633,-0.333333
11614,200005,"""2000-05-01""","""2000-05-31""",4.5390778,-0.16542,-0.109444,-0.414755,-0.102482,1.0202855,-2.210228,1.334411,0.729945,0.392985,0.0262504,-2.014216,0.188484,359.85526,-1.043433,-0.1758,0.5228461,0.12408,-0.105743,-0.053968,0.0,0.289844,0.4865889,-1.0,-0.231268,-0.035137,-0.022064,-0.270667,0.0,-0.270667,0.0,-1.0,-1.839755,…,-1.226052,-0.2

In [121]:
newfile = "NEW_agriculture"
final.write_csv(f'data/industry_datasets/{newfile}.csv')
